In [28]:
import tensorflow as tf
import os

In [29]:
os.chdir("E:\Ineuron\Project\Deep Learning Project\DeepCNNClassifier\DEEPCNNCLASSIFIER")

In [30]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/10tanmay100/DEEPCNNCLASSIFIER.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="10tanmay100"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6017c5edeb1bc23d7d8d742465cc61690dacaad3"

In [31]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [32]:
from dataclasses import dataclass
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [33]:
@dataclass(frozen=True)
class Evaluation_config:
    model_path:Path
    remote_uri:str
    params:dict
    training_data:Path
    params_image_size:list
    params_batch_size:int

In [34]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories,save_json
import tensorflow as tf


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> Evaluation_config:
        return Evaluation_config(model_path=self.config.training.trained_model_path,remote_uri="https://dagshub.com/10tanmay100/DEEPCNNCLASSIFIER.mlflow",params=self.params,training_data="artifacts/data_ingestion/PetImages",params_batch_size=self.params.BATCH_SIZE,params_image_size=self.params.IMAGE_SIZE)

In [35]:
import os
import tensorflow as tf
import time
import urllib.request as request
from zipfile import ZipFile
from deepClassifier.entity import TrainingConfig
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config: Evaluation_config):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(model_path: Path)->tf.keras.Model:
        return tf.keras.models.load_model(model_path)


    def evaluation(self):
        self.model=self.load_model(self.config.model_path)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
    def save_score(self):
        result={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(Path("scores.json"),data=result)
        
    def log_into_mlflow(self):
        remote_uri=self.config.remote_uri
        mlflow.set_tracking_uri(remote_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        mlflow.log_params(self.config.params)
        mlflow.log_metrics({"loss":self.score[0],"accuracy":self.score[1]})
        # Model registry does not work with file store
        if tracking_url_type_store != "file":

            # Register the model
            # There are other ways to use the Model Registry, which depends on the use case,
            # please refer to the doc for more information:
            # https://mlflow.org/docs/latest/model-registry.html#api-workflow
            mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
        else:
            mlflow.keras.log_model(self.model, "model")
        

        

In [36]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    eval=Evaluation(evaluation_config)
    eval.evaluation()
    eval.save_score()
    eval.log_into_mlflow()
except Exception as e:
    raise e

[2022-09-28 18:05:52,493: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-28 18:05:52,501: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-28 18:05:52,504: INFO: common]: created directory at: artifacts
Found 7498 images belonging to 2 classes.
469/469 [==============================] - 1185s 3s/step - loss: 3.2728 - accuracy: 0.7858
[2022-09-28 18:25:39,461: INFO: common]: json file saved at: scores.json
[2022-09-28 18:25:39,494: WARNING: connectionpool]: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ProtocolError('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))': /10tanmay100/DEEPCNNCLASSIFIER.mlflow/api/2.0/mlflow/runs/log-batch


MlflowException: API request to endpoint /api/2.0/mlflow/runs/log-batch failed with error code 400 != 200. Response body: '"repo not associated with run"'